In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [3]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_sensex(f_today):

    days_to_thursday = (4 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [4]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/nifty_min_2019-2024 (1).csv")
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.head()

,index,datetime,o,h,l,c,v
0,nifty,2017-01-02 09:15:00,8210.1000,8211.7000,8189.0000,8189.5500,0
1,nifty,2017-01-02 09:16:00,8188.7500,8193.9500,8188.7500,8189.9500,0
2,nifty,2017-01-02 09:17:00,8190.1500,8190.7500,8173.7000,8173.7000,0
3,nifty,2017-01-02 09:18:00,8173.3500,8177.5500,8169.1500,8177.5500,0
4,nifty,2017-01-02 09:19:00,8177.8500,8178.1500,8173.4500,8174.4000,0


In [5]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
# HL -> High Level
# LL -> Low Level
# PDH -> Previous Day High
# PDL -> Previous Day Low

# Variation 1:
## Entry : HL / LL , SL : PDH / PDL , Condition: Spot breaking HL/LL

# Variation 2:
## Entry : ATM at 9:15 open, SL: PDH / PDL , Condition : Spot breaking HL/LL

# Variation 3:
## Entry: ATM at 9:15 open, SL : HL, LL

# Variation 4:
## Entry : ATM , SL : x% above Entry Premium

In [33]:
async def send_straddle(df):
    results = []
    portfolio_value = 1000000
    index_leverage = 8

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    start_date = dt.date(2024, 11, 1)
    end_date = dt.date(2024, 12, 31)

    current_date = start_date

    while current_date <= end_date:
        print(current_date)
        current_day_data = df[df['datetime'].dt.date == current_date]
        if len(current_day_data) > 0:
            # print(current_day_data.tail())
            expiry = await get_expiry_nifty(current_date)
            dte = (expiry - current_date).days
            atm_strike_ce = int(round(current_day_data['o'].iloc[0] * 1.01 / 50) * 50)
            atm_strike_pe = int(round(current_day_data['o'].iloc[0] * 0.99 / 50) * 50)
            # contract = await get_option_contract_name(
            #     symbol=index_name,
            #     strike=atm_strike,
            #     expiry=expiry,
            #     opt_type="PE",
            # )
            
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike_ce,
                asset_class="C",
            )
            
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike_pe,
                asset_class="P",
            )

            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price_ce = ce_df.iloc[0]["c"]
                exit_price_ce = ce_df.iloc[-1]["c"]

                max_price_ce = ce_df["c"][1:].max()  # Exclude the entry price (row 0)
                min_price_ce = ce_df["c"][1:].min()  # Exclude the entry price (row 0)
            else:
                entry_price_ce = float("nan")
                exit_price_ce = float("nan")
                max_price_ce = float("nan")
                min_price_ce = float("nan")

            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price_pe = pe_df.iloc[0]["c"]
                exit_price_pe = pe_df.iloc[-1]["c"]

                max_price_pe = pe_df["c"][1:].max()  # Exclude the entry price (row 0)
                min_price_pe = pe_df["c"][1:].min()  # Exclude the entry price (row 0)
            else:
                entry_price_pe = float("nan")
                exit_price_pe = float("nan")
                max_price_pe = float("nan")
                min_price_pe = float("nan")

            slippage_ce = 0.01 * (entry_price_ce + exit_price_ce)
            slippage_pe = 0.01 * (entry_price_pe + exit_price_pe)
            
            qty_ce = portfolio_value * index_leverage / atm_strike_ce
            qty_pe = portfolio_value * index_leverage / atm_strike_pe
            points_ce = (entry_price_ce - exit_price_ce)
            points_pe = (entry_price_pe - exit_price_pe)
            pnl_ce = qty_ce * points_ce
            pnl_pe = qty_pe * points_pe
            
            ce_trade = {
                'Day': current_date,
                'Strike': atm_strike_ce,
                'Expiry': expiry,
                'DTE': dte,
                'Option Type': "CE",
                'Entry Price': entry_price_ce,
                'Exit Price': exit_price_ce,
                'Max Price After Entry': max_price_ce,  # Add max price
                'Min Price After Entry': min_price_ce,  # Add min price
                'Points': points_ce,
                'Qty': qty_ce,
                'Pnl': pnl_ce,
                'Slippage': slippage_ce,
                'Final PnL': pnl_ce - slippage_ce,
                'ROI%': (pnl_ce - slippage_ce) * 100 / portfolio_value,
            }
            results.append(ce_trade)

            pe_trade = {
                'Day': current_date,
                'Strike': atm_strike_pe,
                'Expiry': expiry,
                'DTE': dte,
                'Option Type': "PE",
                'Entry Price': entry_price_pe,
                'Exit Price': exit_price_pe,
                'Max Price After Entry': max_price_pe,  # Add max price
                'Min Price After Entry': min_price_pe,  # Add min price
                'Points': points_pe,
                'Qty': qty_pe,
                'Pnl': pnl_pe,
                'Slippage': slippage_pe,
                'Final PnL': pnl_pe - slippage_pe,
                'ROI%': (pnl_pe - slippage_pe) * 100 / portfolio_value,
            }
            results.append(pe_trade)
            
        current_date += dt.timedelta(days=1)

    return pd.DataFrame(results)

tb = await send_straddle(bnf_1min)

2024-11-01
2024-11-02
2024-11-03
2024-11-04
2024-11-05
2024-11-06
2024-11-07
2024-11-08
2024-11-09
2024-11-10
2024-11-11
2024-11-12
2024-11-13
2024-11-14
2024-11-15
2024-11-16
2024-11-17
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29
2024-11-30
2024-12-01
2024-12-02
2024-12-03
2024-12-04
2024-12-05
2024-12-06
2024-12-07
2024-12-08
2024-12-09
2024-12-10
2024-12-11
2024-12-12
2024-12-13
2024-12-14
2024-12-15
2024-12-16
2024-12-17
2024-12-18
2024-12-19
2024-12-20
2024-12-21
2024-12-22
2024-12-23
2024-12-24
2024-12-25
2024-12-26
2024-12-27
2024-12-28
2024-12-29
2024-12-30
2024-12-31


In [129]:
async def send_straddle_w_sl(df, pct, strike_away_pct):
    results = []
    portfolio_value = 1000000
    index_leverage = 8
    RPT_ = 2

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2024, 12, 31)

    current_date = start_date

    while current_date <= end_date:
        print(current_date)
        current_day_data = df[df['datetime'].dt.date == current_date]
        if len(current_day_data) > 0:
            expiry = await get_expiry_nifty(current_date)
            dte = (expiry - current_date).days
            atm_strike_ce = int(round(current_day_data['o'].iloc[0] * (1+(strike_away_pct/100)) / 50) * 50)
            atm_strike_pe = int(round(current_day_data['o'].iloc[0] * (1-(strike_away_pct/100)) / 50) * 50)

            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike_ce,
                asset_class="C",
            )
            
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=current_date,
                end_date=current_date,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike_pe,
                asset_class="P",
            )

            # Process CE data
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price_ce = ce_df.iloc[0]["c"]
                sl_price_ce = entry_price_ce * (1+pct)
                exit_price_ce = ce_df.iloc[-1]["c"]
                exit_time_ce = ce_df.iloc[-1]["datetime"]

                # Check if SL is hit during the session
                for _, row in ce_df.iterrows():
                    if row["c"] >= sl_price_ce:
                        exit_price_ce = sl_price_ce
                        exit_time_ce = row["datetime"]
                        break

                max_price_ce = ce_df["c"][1:].max()
                min_price_ce = ce_df["c"][1:].min()
            else:
                entry_price_ce = exit_price_ce = max_price_ce = min_price_ce = sl_price_ce = float("nan")
                exit_time_ce = None

            # Process PE data
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price_pe = pe_df.iloc[0]["c"]
                sl_price_pe = entry_price_pe * (1+pct)
                exit_price_pe = pe_df.iloc[-1]["c"]
                exit_time_pe = pe_df.iloc[-1]["datetime"]

                # Check if SL is hit during the session
                for _, row in pe_df.iterrows():
                    if row["c"] >= sl_price_pe:
                        exit_price_pe = sl_price_pe
                        exit_time_pe = row["datetime"]
                        break

                max_price_pe = pe_df["c"][1:].max()
                min_price_pe = pe_df["c"][1:].min()
            else:
                entry_price_pe = exit_price_pe = max_price_pe = min_price_pe = sl_price_pe = float("nan")
                exit_time_pe = None

            # Calculate slippage, qty, and PnL
            slippage_ce = 0.01 * (entry_price_ce + exit_price_ce)
            slippage_pe = 0.01 * (entry_price_pe + exit_price_pe)
            
            # qty_ce = portfolio_value * index_leverage / atm_strike_ce
            # qty_pe = portfolio_value * index_leverage / atm_strike_pe

            qty_ce = (RPT_ / 100) * portfolio_value / abs(entry_price_ce - sl_price_ce)
            qty_pe = (RPT_ / 100) * portfolio_value / abs(entry_price_pe - sl_price_pe)

            if (
                (qty_ce * atm_strike_ce) / (index_leverage * portfolio_value)
            ) * 100 > 250:
                qty_ce = portfolio_value * index_leverage / atm_strike_ce * 2.5

            if (
                (qty_pe * atm_strike_pe) / (index_leverage * portfolio_value)
            ) * 100 > 250:
                qty_pe = portfolio_value * index_leverage / atm_strike_pe * 2.5
            
            points_ce = (entry_price_ce - exit_price_ce)
            points_pe = (entry_price_pe - exit_price_pe)
            pnl_ce = qty_ce * points_ce
            pnl_pe = qty_pe * points_pe
            
            # Record CE trade
            ce_trade = {
                'Day': current_date,
                'Strike': atm_strike_ce,
                'Expiry': expiry,
                'DTE': dte,
                'Option Type': "CE",
                'Entry Price': entry_price_ce,
                'Initial SL': sl_price_ce,
                'Exit Price': exit_price_ce,
                'Exit Time': exit_time_ce,  # Add exit time
                'Max Price After Entry': max_price_ce,
                'Min Price After Entry': min_price_ce,
                'Points': points_ce,
                'Qty': qty_ce,
                'Pnl': pnl_ce,
                'Slippage': slippage_ce,
                'Final PnL': (points_ce - slippage_ce) * qty_ce,
                "Margin": ((qty_ce * atm_strike_ce) / (index_leverage * portfolio_value))* 100,
                'ROI%': (points_ce - slippage_ce) * qty_ce * 100 / portfolio_value,
            }
            results.append(ce_trade)

            # Record PE trade
            pe_trade = {
                'Day': current_date,
                'Strike': atm_strike_pe,
                'Expiry': expiry,
                'DTE': dte,
                'Option Type': "PE",
                'Entry Price': entry_price_pe,
                'Initial SL': sl_price_pe,
                'Exit Price': exit_price_pe,
                'Exit Time': exit_time_pe,  # Add exit time
                'Max Price After Entry': max_price_pe,
                'Min Price After Entry': min_price_pe,
                'Points': points_pe,
                'Qty': qty_pe,
                'Pnl': pnl_pe,
                'Slippage': slippage_pe,
                'Final PnL': (points_pe - slippage_pe) * qty_pe,
                "Margin": ((qty_pe * atm_strike_pe) / (index_leverage * portfolio_value))* 100,
                'ROI%': (points_pe - slippage_pe) * qty_pe * 100 / portfolio_value,
            }
            results.append(pe_trade)
            
        current_date += dt.timedelta(days=1)

    return pd.DataFrame(results)

In [131]:
tb = await send_straddle_w_sl(bnf_1min, 0.3, 1)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01

In [126]:
pct_range = [0.2, 0.3, 0.4, 0.5]
strike_away_pct_range = [0, 0.5, 1, 1.5, 2, 2.5]
for i in pct_range:
    for j in strike_away_pct_range:
        print(f'SL PCT: {i}%, Strike Away: {j}%')
        tb = await send_straddle_w_sl(bnf_1min, i, j)
        if len(tb)> 0:
            tradebook = tb
            tradebook["Day"] = pd.to_datetime(tradebook["Day"])
            tradebook["Trade Year"] = tradebook["Day"].dt.year
            stats = generate_stats(tradebook, f'SL PCT: {i}%, Strike Away: {j}%')
            for x, y in stats.items():
                print(y.to_string())
                # break
            
        # pdx

SL PCT: 0.2%, Strike Away: 0%
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                      Variation
2017      43.4088          496   8.0645                3.8489             -1.5793     -10.2175       4.2485  SL PCT: 0.2%, Strike Away: 0%
2018      18.6926          492   6.7073                4.7962             -1.7232     -23.8503       0.7837  SL PCT: 0.2%, Strike Away: 0%
2019     130.0032          490  30.6122                4.3469             -1.8126     -32.3243       4.0218  SL PCT: 0.2%, Strike Away: 0%
2020     275.4369          504  37.6984                4.6106             -1.9373     -38.5733       7.1406  SL PCT: 0.2%, Strike Away: 0%
2021     253.5466          496  36.4919                4.6187             -1.8728     -35.5575       7.1306  SL PCT: 0.2%, Strike Away: 0%
2022     215.8342          496  36.4919                4.5297             -1.9298     -20.0084      10.7872  SL PCT: 0.2%, Strike Away: 

In [132]:
tb['Points'].sum()

10581.589999999998

In [133]:
tradebook = tb
tradebook["Day"] = pd.to_datetime(tradebook["Day"])
tradebook["Trade Year"] = tradebook["Day"].dt.year

In [136]:
def generate_stats(tb, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2019, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [137]:
stats = generate_stats(tradebook, '...')
for x, y in stats.items():
    pdx = pd.DataFrame(y)
    break

pdx

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,123.8705,490,41.6327,1.8073,-1.0463,-15.0218,8.2461,...
2020,204.2663,504,47.8175,2.4474,-1.4886,-14.4131,14.1723,...
2021,196.1106,496,48.7903,2.0744,-1.2235,-14.5699,13.4600,...
2022,139.2202,496,47.9839,2.1114,-1.4191,-13.2383,10.5165,...
2023,82.8390,492,48.1707,1.0668,-0.6995,-7.7167,10.7350,...
2024,129.4651,498,43.1727,1.8449,-1.0316,-11.4248,11.3319,...
Overall,875.7717,2976,46.2702,1.8972,-1.1570,-15.0218,58.3002,...


In [84]:
# tradebook['Price Spike'] = tradebook['Max Price After Entry'] / tradebook['Entry Price']
# tradebook_pe['Price Spike'] = tradebook_pe['Max Price After Entry'] / tradebook_pe['Entry Price']

In [106]:
# tradebook['ROI%'].sum()
tb2 = tradebook[(tradebook['Entry Price'] < 5)]
tb2['ROI%'].sum()

37.314607392916386

In [140]:
tb_m = tradebook[tradebook['Margin'] >= 250]
len(tb_m)

1918

In [60]:
tradebook.tail(50)

,Day,Strike,Expiry,DTE,Option Type,Entry Price,Exit Price,Exit Time,Max Price After Entry,Min Price After Entry,Points,Qty,Pnl,Slippage,Final PnL,ROI%,Trade Year
3914,2024-11-26,24600,2024-11-28,2,CE,27.3500,34.1875,2024-11-26 09:22:00,38.7500,8.1000,-6.8375,1462.5229,-10000.0000,0.6154,-10000.6154,-1.0001,2024
3915,2024-11-26,24100,2024-11-28,2,PE,66.1000,82.6250,2024-11-26 10:16:00,104.7500,48.0500,-16.5250,605.1437,-10000.0000,1.4872,-10001.4873,-1.0001,2024
3916,2024-11-27,24450,2024-11-28,1,CE,21.7500,27.1875,2024-11-27 12:21:00,51.3000,11.9000,-5.4375,1839.0805,-10000.0000,0.4894,-10000.4894,-1.0000,2024
3917,2024-11-27,23950,2024-11-28,1,PE,30.4500,5.8500,2024-11-27 15:30:00,28.3500,5.6500,24.6000,1313.6289,32315.2709,0.3630,32314.9079,3.2315,2024
3918,2024-11-28,24500,2024-11-28,0,CE,13.1500,16.4375,2024-11-28 09:18:00,24.4500,0.0500,-3.2875,3041.8251,-10000.0000,0.2959,-10000.2959,-1.0000,2024
3919,2024-11-28,24050,2024-11-28,0,PE,9.9000,12.3750,2024-11-28 09:16:00,221.8000,4.2000,-2.4750,4040.4040,-10000.0000,0.2228,-10000.2228,-1.0000,2024
3920,2024-11-29,24150,2024-12-05,6,CE,127.4500,159.3125,2024-11-29 09:33:00,214.4000,125.7000,-31.8625,313.8486,-10000.0000,2.8676,-10002.8676,-1.0003,2024
3921,2024-11-29,23700,2024-12-05,6,PE,75.9500,28.1500,2024-11-29 15:30:00,86.5500,27.3000,47.8000,526.6623,25174.4569,1.0410,25173.4159,2.5173,2024
3922,2024-12-02,24400,2024-12-05,3,CE,42.8500,53.5625,2024-12-02 09:40:00,127.4500,30.3500,-10.7125,933.4889,-10000.0000,0.9641,-10000.9641,-1.0001,2024
3923,2024-12-02,23900,2024-12-05,3,PE,86.6500,43.0000,2024-12-02 15:30:00,97.6000,38.5000,43.6500,461.6272,20150.0289,1.2965,20148.7324,2.0149,2024


In [60]:
percentiles = tradebook_pe['Price Spike'].quantile([0.25, 0.5, 0.75, 0.9, 0.95])

print(percentiles)

0.2500   1.0740
0.5000   1.3218
0.7500   1.9274
0.9000   3.0230
0.9500   4.6326
Name: Price Spike, dtype: float64


In [55]:
tradebook2 = tradebook.sort_values(by='Hypo ROI% Max')
tradebook2.head(100)

,Day,Strike,Expiry,DTE,Option Type,Entry Price,Exit Price,Max Price After Entry,Min Price After Entry,Qty,Pnl,Slippage,Final PnL,ROI%,Trade Year,Price Spike,Hypo ROI% Max
1580,2020-03-13,9200,2020-03-19,6,CE,158.4000,920.1000,1031.0500,151.3000,869.5652,-662347.8261,10.7850,-662358.6111,-66.2359,2020,6.5092,-75.8826
3677,2024-06-04,22950,2024-06-06,2,PE,427.9500,1152.3500,1824.1000,370.3500,348.5839,-252514.1612,15.8030,-252529.9642,-25.2530,2024,4.2624,-48.6675
1342,2019-09-20,10850,2019-09-26,6,CE,37.5000,458.9500,570.2500,34.5000,737.3272,-310746.5438,4.9645,-310751.5083,-31.0752,2019,15.2067,-39.2811
1590,2020-03-20,8350,2020-03-26,6,CE,331.5000,593.0000,730.9000,326.5500,958.0838,-250538.9222,9.2450,-250548.1672,-25.0548,2020,2.2048,-38.2659
1579,2020-03-12,9950,2020-03-12,0,PE,8.4500,359.5000,462.6000,15.1000,804.0201,-282251.2563,3.6795,-282254.9358,-28.2255,2020,54.7456,-36.5146
1587,2020-03-18,9000,2020-03-19,1,PE,207.9500,581.9500,609.9000,177.0000,888.8889,-332444.4444,7.8990,-332452.3434,-33.2452,2020,2.9329,-35.7289
1588,2020-03-19,8150,2020-03-19,0,CE,51.0500,111.1000,404.0000,25.0500,981.5951,-58944.7853,1.6215,-58946.4068,-5.8946,2020,7.9138,-34.6454
1596,2020-03-25,7800,2020-03-26,1,CE,359.2000,598.3500,669.5500,242.7500,1025.6410,-245282.0513,9.5755,-245291.6268,-24.5292,2020,1.8640,-31.8308
1610,2020-04-07,8550,2020-04-09,2,CE,90.6500,374.8500,374.8500,64.3500,935.6725,-265918.1287,4.6550,-265922.7837,-26.5923,2020,4.1351,-26.5918
1967,2020-12-21,13600,2020-12-24,3,PE,55.4000,343.2500,455.9500,32.8500,588.2353,-169323.5294,3.9865,-169327.5159,-16.9328,2020,8.2301,-23.5618


In [54]:
tradebook['Hypo ROI% Max'] = (tradebook['Entry Price'] - tradebook['Max Price After Entry']) * tradebook['Qty'] * 100 / 1000000

In [49]:
tradebook_ce = tradebook[tradebook['Option Type'] == 'CE']
tradebook_pe = tradebook[tradebook['Option Type'] == 'PE']

In [141]:
tradebook.to_csv('Lobster_1_30.csv', index=False)